In [1]:
from pymatgen import MPRester
import pandas as pd
from pymatgen import Structure
import json
m = MPRester('lh8Nu9sPc6U1Vxkn')

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
class MaterialsMiner():
    def __init__(self):
        pass
    def materials_class(self):
        '''
        This function allows one to define the family of materials that they are interested in and subsequently collect their material IDs and formulas. 
        perovskites, anti-perovskites, transition metal nitrides and transition metal carbides have been provided for starters, 
        More families can be added as one desires. 
        '''

        perovskites = m.query(criteria={'formula_anonymous':"ABC3", 'elements':{'$in':['Ba', 'Sr', 'Ca'], "$all":['Ti', 'O']}}, properties = ['task_id', 'pretty_formula'])
        anti_perovskites = m.query(criteria={'formula_anonymous':'A3BC', 'elements':{'$in':['Ba', 'Sr', 'Ca'], "$all":['Sn', 'O']}}, properties=['task_id', 'pretty_formula'])
        tmns = m.query(criteria = {'formula_anonymous':'AB', 'elements':{'$in':['Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru','Rh', 'Rb', 'Pd', 'Ag', 'Cd'], "$all":['N']}}, properties=['task_id', 'pretty_formula'])
        tmcs = m.query(criteria={'formula_anonymous':'AB', 'elements':{'$in':['Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rb','Rh', 'Pd', 'Ag', 'Cd'], "$all":['C']}}, properties=['task_id', 'pretty_formula'])
        perovskites_df, anti_perovskites_df, tmns_df, tmcs_df = pd.DataFrame(perovskites), pd.DataFrame(anti_perovskites), pd.DataFrame(tmns), pd.DataFrame(tmcs)
        return perovskites_df, anti_perovskites_df, tmns_df, tmcs_df
    def get_properties(self,family):
        '''
        This function takes in a family of perovskites from the data frames created in the materialsclass function. 
        It then iterates through every id in the dataframe and obtains the final energy, the bandgap, density, energy and possion ratio
        
        '''
        family_ids = family.iloc[:, 0].values
        fenergy=[];  gap=[]; density=[] ; energy_p=[] ;  ratio=[]
        for num, i in enumerate(family_ids):

            properties = m.query(criteria={'task_id': i}, properties=['final_energy', 'band_gap', 'density', 'energy_per_atom', 'poisson_ratio'])
            fenergy.append(properties[0]['final_energy'])
            gap.append(properties[0]['band_gap'])
            density.append(properties[0]['density'])
            energy_p.append(properties[0]['energy_per_atom'])
            ratio.append(properties[0]['poisson_ratio'])
        family['final_energy'] = fenergy
        family['band_gap'] = gap
        family['density'] = density
        family['energy'] = energy_p
        family['poisson_ratio'] = ratio
        return family
    def save_to_csv(self,family):
        '''
        This function simply takes the dataframe created from the get_properties function and saves it as a csv file for any further use
        '''
        family_name = family
        csv_file = family.to_csv('family.csv') #This will be edited depending on the material family being used
        return csv_file


In [3]:
mat_miner = MaterialsMiner()
perovs, anti_perovs, tmns, tmcs = mat_miner.materials_class()
tmns.head()


,task_id,pretty_formula
0,mvc-14624,MoN
1,mp-1009548,PdN
2,mp-1065394,MoN
3,mp-1067644,PdN
4,mp-13126,ZrN


In [4]:
tmcs.head()

,task_id,pretty_formula
0,mp-910,NbC
1,mp-1009837,TcC
2,mp-1009878,ZrC
3,mp-10030,RuC
4,mp-1009652,RhC


In [5]:
tmns_df = mat_miner.get_properties(tmns)
tmcs_df = mat_miner.get_properties(tmcs)
deep_learning_data = pd.concat([tmns_df, tmcs_df]).reset_index(drop=True)
deep_learning_data.head()

,task_id,pretty_formula,final_energy,band_gap,density,energy,poisson_ratio
0,mvc-14624,MoN,-38.807335,0.0,7.646723,-9.701834,None
1,mp-1009548,PdN,-11.250518,0.0,9.122720,-5.625259,None
2,mp-1065394,MoN,-39.775364,0.0,8.966646,-9.943841,None
3,mp-1067644,PdN,-21.571676,0.0,4.474094,-5.392919,None
4,mp-13126,ZrN,-38.693013,0.0,5.639729,-9.673253,None


In [7]:
deep_learning_data.columns

Index(['task_id', 'pretty_formula', 'final_energy', 'band_gap', 'density',
       'energy', 'poisson_ratio'],
      dtype='object')

In [14]:
ids = deep_learning_data['task_id'].values
structures = []
for num, i in enumerate(ids):
    structure = m.get_structure_by_material_id(i)
    structures.append(structure)


In [15]:
with open('structures.txt', 'w') as f:
    for item in structures:
        f.write("%s\n" % item)

In [16]:
deep_learning_data.to_csv('transition_metal_carbides_nitrides.csv')